In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import numpy as np

In [2]:
data = pd.read_csv('dataset/Churn_Modelling.csv')

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [5]:
## Encode Categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [6]:
## One hot encoder for "Geography"
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [7]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [9]:
# Combine the one-hot encoded columns with the original dataframe
# and drop the original 'Geography' column
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
#split the data into features and target variable
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [11]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
# import pickle
# #save the encoders and scalers for later use
# with open('reg_label_encoder_gender.pkl', 'wb') as file:
#     pickle.dump(label_encoder_gender, file)

# with open('reg_onehot_encoder_geo.pkl', 'wb')as file:
#     pickle.dump(onehot_encoder_geo, file)

# with open('reg_scaler.pkl', 'wb') as file:
#     pickle.dump(scaler, file)
          

## ANN Regression Problem statement

In [15]:
import pickle
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [16]:
## Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    # Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

## compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

C:\Users\Nitin Mishra\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
import datetime

#Set up TensorBoard callback
log_dir = "Regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [19]:
#set up early stopping callback
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [20]:
history = model.fit(X_train, y_train, epochs=100, 
                    batch_size=32, 
                    validation_split=0.2,
                    validation_data=(X_test, y_test),
                    verbose=1,
                    callbacks=[tensorboard_callback, early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 13346556928.0000 - mae: 100002.8516 - val_loss: 13010099200.0000 - val_mae: 98533.2500
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 13356978176.0000 - mae: 100169.1328 - val_loss: 12738417664.0000 - val_mae: 97155.6484
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 12977930240.0000 - mae: 98375.4688 - val_loss: 12035906560.0000 - val_mae: 93595.2031
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 12192573440.0000 - mae: 94554.9297 - val_loss: 10841653248.0000 - val_mae: 87534.7109
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 10627980288.0000 - mae: 86710.4844 - val_loss: 9255101440.0000 - val_mae: 79554.0703
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 9019437056.0000 - mae: 78350.5078 - val_loss: 7517912064.0000 - val_mae: 70905.4219
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 7194306048.0000 - mae: 69399.2266 - val_loss: 5939456512.0

In [29]:
%load_ext tensorboard
%tensorboard --logdir Regressionlogs/fit/20250314-153036

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
